# Objects Detection using SSD

### Importing the libraries

In [35]:
import torch
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio
import cv2

### Detection Function

In [36]:
def detect(frame, net, transform):
    height, width = frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2, 0, 1) # permute the dimensions of the image to match the input of the network
    x = x.unsqueeze(0) # add a fake dimension corresponding to the batch
    
    with torch.no_grad():
        y = net(x) # feed the image to the network to get the output
    
    detections = y.data # extract the detections from the network output
    scale = torch.Tensor([width, height, width, height]) # create a tensor object with the scale
    for i in range(detections.size(1)): # loop over the number of detections
        j = 0 # initialize the loop variable
        while detections[0, i, j, 0] >= 0.6: # loop over the detections with a confidence higher than 0.6
            pt = (detections[0, i, j, 1:] * scale).numpy() # extract the coordinates of the detected object
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255,0,0), 2) # draw a rectangle around the detected object
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA) # add a label around the detected object
            j += 1
        
    return frame    

### SSD Neural Network

In [37]:
with torch.no_grad():
    net = build_ssd('test') # create the network
    net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage)) # load the weights of the pre-trained network

### Transformations

In [38]:
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0)) # create the transformation

### Detections on the Video

In [39]:
reader = imageio.get_reader('/home/arslanhaider/ahs-data/epic_horses.mp4') # open the video
fps = reader.get_meta_data()['fps'] # get the fps frequence (frames per second)
writer = imageio.get_writer('/home/arslanhaider/ahs-data/output.mp4', fps = fps) # create an output video with this same fps frequence

# loop over all frames of the video
for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform) # detect objects in the frame
    writer.append_data(frame) # add the frame to the output video
    print(i) # print the number of the processed frame

RuntimeError: Legacy autograd function with non-static forward method is deprecated. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)